<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Keras_WithDeeplizard/blob/master/12_Build_And_Train_A_Convolutional_Neural_Network_With_TensorFlow's_Keras_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build And Train A Convolutional Neural Network With TensorFlow's Keras API
在本集中，我们将演示如何构建一个简单的卷积神经网络(CNN)，并使用TensorFlow的Keras API对猫和狗的图像进行训练。

我们将使用我们在上一集准备的图像数据。一定要确保你已经完成了那个章节的第一步来获取和准备数据，同时也要确保你还有我们上次引入的所有导入，因为我们会继续在这里使用它们。

## 1. 构建一个简单的CNN
为了构建CNN，我们将使用Keras顺序模型。回想一下，我们在前面的章节中首先介绍了顺序模型。

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
model = Sequential([
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same',input_shape=(224,224,3)),
    # strides：步幅
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    MaxPool2D(pool_size=(2,2),strides=2),
    Flatten(),
    Dense(units=2,activation='softmax')
])

模型的第一层是一个二维卷积层。这一层将有32个输出滤波器，每个滤波器的内核大小为3x3，我们将使用relu激活函数。

**注意：**指定的输出滤波器数量的选择是任意的，所选择的核大小3x3通常是非常常用的大小。您可以通过为这些参数选择不同的值来进行试验。

我们通过指定padding = 'same'来启用零填充。

仅在第一层，我们还指定了输入形状，也就是数据的形状。我们的图像是224像素高和224像素宽，有3个颜色通道:RGB。这给了我们一个输入形状(224,224,3)。

然后我们添加一个最大池层池和减少数据的维数。注意，为了获得对最大池化、零填充、卷积滤波器和卷积神经网络的基本理解，请查看深度学习基础课程。

在此之后，我们添加另一个卷积层，其规格与前面的层完全相同，只是第二个Conv2D层有64个过滤器。这里选择64也是任意的，但是通常的选择是在后面的层中使用比前面的层更多的过滤器。这个层后面是相同类型的MaxPool2D层。

然后我们将卷积层的输出压平，并将其传递到一个密集层。这个密集的层是网络的输出层，所以它有两个节点，一个为猫，一个为狗。我们将在输出上使用softmax激活函数，以便每个样本的输出是cat和dog输出的概率分布。

我们可以通过调用model.summary()来查看模型的摘要。

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 200704)            0         
_________________________________________________________________
dense (Dense)                (None, 2)                 401410    
Total params: 420,802
Trainable params: 420,802
Non-trainable params: 0
__________________________________________________

现在模型已经建立，我们使用Adam优化器编译模型，学习率为0.0001，分类交叉熵损失，我们将把准确性作为性能指标。同样，如果你需要对这些主题的基本理解，请查看深度学习基础课程。

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])

**注意:**当我们只有两个类时，我们可以将输出层配置为只有一个输出，而不是两个输出，并使用binary_crossentropy作为损失，而不是categorical_crossentropy。 两种选择均能很好地工作，并获得完全相同的结果。

但是，对于binary_crossentropy，最后一层需要使用Sigmoid而不是softmax作为其激活函数。

## 2. 训练一个简单的CNN
首先导入前几集构建的数据集

In [ ]:
# 在colab上使用的云端数据
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = "/content/drive/My Drive/DeepLearning_AI_Course/Deeplizard/Keras - Python Deep Learning Neural Network API/Data/dogs-vs-cats"

# 更改当前系统的路径
os.chdir(path)
# 显示当前路径的文件夹名字
print(os.listdir())

# 获得当前路径
os.getcwd()

['Dog', 'Cat', 'train', 'valid', 'test']


'/content/drive/My Drive/DeepLearning_AI_Course/Deeplizard/Keras - Python Deep Learning Neural Network API/Data/dogs-vs-cats'

In [ ]:
os.chdir('..')
print(os.listdir())
train_path = './dogs-vs-cats/train'
valid_path = './dogs-vs-cats/valid'
test_path = './dogs-vs-cats/test'
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

['dogs-vs-cats']
Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


现在是时候训练模型了。

我们已经引入了model.fit（）函数来训练上一集中的模型。 除了现在，我们将以相同的方式使用它，除了现在，我们将传入新引入的DirectoryIterators train_batches和valid_batches来训练和验证模型。 回想一下，这些是在上一集中创建的。

In [ ]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=10,
    verbose=2
)

Epoch 1/10


UnknownError: ignored

我们需要指定steps_per_epoch来指示在宣布一个时期完成之前，应将训练集中的多少批次样本传递给模型。 由于我们的训练集中有1000个样本，并且批次大小为10，所以我们将steps_per_epoch设置为100，因为100个批次（每10个样本）将涵盖我们的整个训练集。

我们可以使用len（train_batches）作为指定此值的更通用的方法，因为train_batches的长度等于100，因为它由100个批次的10个样本组成。 同样，我们以相同的方式指定validate_steps，但使用validate_batches。

我们指定10作为我们想要运行的epoch的数量，并将verbose参数设置为2，这只是指定在训练期间打印到控制台的日志输出的详细程度。

从上面的输出中，我们可以看到这个简单模型在训练集上的性能很好，准确率达到100%，loss接近于0，但是，通过将这些结果与验证指标进行比较，我们可以看到我们的模型对训练数据有很大的过拟合。

在这一点上，我们可以继续研究该模型以应对过度拟合，或者可以尝试另一种对数据使用预训练模型的方法。 我们将在下一集中探讨后者！
